# Настройка ноутбука

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

import scipy.stats as st

import plotly.figure_factory as ff
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Расширить рабочее поле ноутбука на весь экран
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Описание

В этом ноутбуке я приводжу туториал по проведению A/B (A/A) тестов.

**Курс:**

**Видео:**


- [Анатолий Карпов - A/B-тестирование: как сделать так, чтобы оно заработало](https://www.youtube.com/watch?v=dFCJysbOJ8c&t=24s&ab_channel=%D0%9C%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%80%D0%BA%D0%B5%D1%82%D0%B8%D0%BD%D0%B3)
- [Как и зачем оценивать размер выборки для A/B теста?](https://www.youtube.com/watch?v=2nP_gcut7SU&ab_channel=karpov.courses)
- [Тонкости A/B тестирования: проблема подглядывания | Вебинар Анатолия Карпова | karpov.courses](https://www.youtube.com/watch?v=jnFVmtaeSA0&ab_channel=karpov.courses)
- [Никита Маршалкин: А/Б тесты сложнее, чем кажется | Интервью | karpov.courses](https://www.youtube.com/watch?v=gljfGAkgX_o&ab_channel=karpov.courses)
- [A/B-тесты с метриками-отношениями и при чём здесь внутрипользовательские корреляции | karpov.courses](https://www.youtube.com/watch?v=ObzlKVCiBqI&ab_channel=karpov.courses)
- [Medium: Practitioner’s Guide to Statistical Tests](https://vkteam.medium.com/practitioners-guide-to-statistical-tests-ed2d580ef04f)

**Статьи:**

# Формулирование гипотезы

**Что меняем?**

**На что повлияет?**

**Как измеряем?**

**Эффект эксперимента** - различие значений между группами, вызванное экспериментом.

<img src="data/img/02 - effect - 1.png" width="800">
<img src="data/img/02 - effect - 2.png" width="800">

**Статистический критерий** - математическое правило, в соответствии с которым принимается или отвергается статистическая гипотеза

**Ошибка I рода** - сказать, что эффект есть, когда на самом деле его нет. Ложноположительный результат. При ошибке I рода тратим впустую средства на внедрение изменения, которое не имеет эффекта;

**Ошибка II рода** - сказать, что эффекта нет, когда на самом деле он есть. Ложноотрицательный результат. При ошибке II рода упускаем возможность зарабатывать больше, отказываясь внедрять изменение
с эффектом.

**Уровень значимости** - веротность того, что статистический критерий совершит ошибку I рода. Уровень значимости обозначают символом $\alpha$. Иными словами, это допустимый уровень ошибки I рода

**Статистическая мощьность** - вероятность сказать, что эффект есть, когда он на самом деле есть. Статистическая мощность и вероятность ошибки II рода в сумме дают единицу. Чем больше мощность, тем меньше вероятность ошибки II рода и наоборот.

Из всех критериев нужно выбирать тот, который в конкретной задаче при фиксированном уровне значимости показывает наибольшее значение мощности. Такой критерий называют **наиболее мощным** или **наиболее чувствительным**. Иными словами нужно выбирать такой критерий, который при фиксированном уровне значимости показывает наименьшую вероятность ошибки II рода.

**Замечания:** нужно будет опробовать различные варианты визуализаций ошибок I и II родов.

# Статистические критерии

У разных критериев разные формулы и разные распределения. Вот несколько популярных критериев:

- Тест Стьюдента. Проверяет равенство средних значений выборок;
- Тест Манна-Уитни. Проверяет, что две выборки из одного распределения;
- Тест Дики-Фуллера. Проверяет наличие единичного корня в авторегрессионной модели временного ряда первого порядка;
- Тест Колмогорова-Смирнова. Проверяет принадлежность выборки к заранее известному распределению.

Чтобы сравнить результаты разных критериев нужно помнить распределения и пороговые значения разных тестов — это неудобно. Чтобы легче было сравнивать результаты разных критериев можно использовать p-value.

**p-value** - вероятность получить значение статистики как в эксперименте или более экстремальное при условии того, что нулевая гипотеза верна.

Если статистикой эксперимента является разница средних, а в экспериментальной группе среднее на 5 рублей больше, то p-value — это вероятность получить отличие между средними не меньше чем на 5 рублей при условии, что эффекта на самом деле нет.

С помощью p-value легко понимать результат любого статистического критерия:

Если p-value меньше уровня значимости α (обычно берется равным 0.05), то отклоняем нулевую гипотезу, то есть заявляем, что удалось обнаружить значимые отличия;
Если p-value больше уровня значимости α (обычно берется равным 0.05), то данные нулевой гипотезе не противоречат, значимые отличия не найдены.

# Дизайн эксперимента

Вернёмся к составлению дизайна. Что он должен содержать? Можно выделить три основных этапа эксперимента, которые должны быть описаны в дизайне:

1. **Формирование гипотезы.** Определяем, что хотим изменить, на что это повлияет и с помощью какой метрики будем измерять это влияние.
2. **Проведение эксперимента.** Нужно зафиксировать время проведения эксперимента, и то, как будут формироваться контрольная и экспериментальная группы.
3. **Оценка результатов.** Описать, как будут обрабатываться данные и приниматься решение.

## Подготовка эксперимента

### Размер групп

Чем больше размер групп, тем больше данных мы получаем. С одной стороны, это позволит принять более точное решение. 

С другой стороны, это увеличит потери в случае, если эксперимент оказался неудачным. Если в новую версию бэкенда закралась ошибка и сайт перестанет работать у всех пользователей экспериментальной группы, то чем больше размер групп, тем больше людей не смогут сделать заказ. 

И ещё, чем больше размер групп, тем меньше экспериментов мы можем проводить. Это может быть важно, если накопилось много гипотез и хочется побыстрее их все проверить.

<img src="data/img/groups.png" width="800">

<img src="data/img/group_formula.png" width="500">

α − вероятности ошибки первого рода, она же уровень значимости. Мы зафиксировали её в дизайне равной 0.05.

β − вероятности ошибки второго рода, с какой вероятностью мы скажем что эффекта нет, когда на самом деле он есть.

Часто его берут равным 0.1 или 0.2.


σ^x и σ^y − стандартные отклонения в контрольной и экспериментальной группах соответственно. Эти значения можно оценить, подсчитав стандартные отклонения метрики по историческим данным.

ϵ − ожидаемый эффект - эффект, который мы ожидаем получить от эксперимента. В эксперименте с бэкендом ожидаемый эффект может быть, например, 5 миллисекунд. Это означает, что если окажется верна альтернативная гипотеза, и истинный эффект будет 5 миллисекунд или более, то мы обнаружим его
с вероятностью не менее 1 - β.

Φ−1 − это обозначение специальной функции из статистики (обратная функция нормального распределения), её значения можно получить, поменяв для функции распределения горизонтальную и вертикальную ось местами.

При α = 0.05 получаем Φ−1(1 - α/2)=1.96, а при β = 0.1 получаем Φ−1(1 - β) = 1.28.

<img src="data/img/Функция распределения.png" width="600">

Получается, мы можем вычислить размер выборки, если определим допустимые вероятности ошибок, ожидаемый размер эффекта и оценим стандартное отклонение метрики. Рекомендуем использовать размер выборки на 5-10% больше рассчитанного по формуле, так как в формуле используются не точные значения стандартных отклонений, а их оценки, которые имеют погрешность и могут немного искажать результат.

#### Проверка:

- [x] При увеличении стандартного отклонения в 2 раза, необходимый размер групп увеличивается в 4 раза
- [x] При уменьшении вероятности ошибки первого рода, размер выборки увеличивается
- [ ] При уменьшении вероятности ошибки второго рода, размер выборки уменьшается
- [x] Чтобы быть способным поймать меньший эффект, нужно больше данных

<img src="data/img/choose_group.png" width="800">

### MDE и Реальный эффект

**MDE (minimum detectable effect - минимальный детектируемый эффект)** - размер эффекта, который можно обнаружить с заданными вероятностями ошибок, стандартными отклонениями и размером групп.

Конкретного алгоритма определения размера ожидаемого эффекта не существует, но есть некоторые подходы, которые могут быть полезны:

1. Оттолкнуться от минимального размера эффекта, который мы сможем распознать при тесте на всех пользователях. Мы не сможем распознать эффект меньшего размера;
2. Использовать опыт прошедших тестов. Если ранее проводились похожие тесты, можем использовать результаты их оценок для определения целевого эффекта;
3. Исходить из затрат на проведение теста. Например, нет смысла выбирать MDE меньше, чем себестоимость эксперимента.

<img src="data/img/mde_dialog.png" width="400">

Из диалога выше видно, что выбор размера групп — это некоторый компромисс между расходами на проведение эксперимента и точностью полученных результатов. Чтобы понять, какой минимальный эффект мы можем обнаружить для фиксированного размера групп, нужно вычислить MDE.

  
Формула для MDE легко выводится из формулы для размера выборки:

<img src="data/img/mde_formula.png" width="400">

Теперь у нас есть первый способ оценить MDE: посчитать, какой минимальный эффект способны обнаружить, если мы возьмём в эксперимент всех доступных пользователей.
Допустим, MDE оказался равен 100 рублей, а наш эксперимент больше чем на 10 рублей среднюю выручку
не увеличит. Мы не сможем обнаружить эффект такого размера, что же делать? Нужно попробовать поменять дизайн эксперимента с целью получить приемлемое значением MDE. Для этого можно попробовать увеличить продолжительность эксперимента, чтобы в нем успели поучаствовать больше пользователей, выбрать другую метрику или применить способы снижения дисперсии данных.

Если не удалось получить приемлемое значение MDE, то от проведения эксперимента с таким дизайном лучше отказаться, так как обнаружить эффект, скорее всего, не удастся.

Рассмотрим другую ситуацию: MDE для эксперимента на всех пользователях оказался равен 1 рублю, наш эксперимент потенциально может показать такой эффект. Как в этом случае выбрать параметры для эксперимента? 

Мы можем предварительно оценить ожидаемый в эксперименте эффект. Если у нас уже есть опыт запуска подобных экспериментов, посмотрим, какие в них были эффекты, затем оценим значение ожидаемого эффекта, исходя из имеющихся данных.

Также можно оттолкнуться от затрат на внедрение изменений. Например, если отправка рекламного письма стоит 2 рубля, необходимо, чтобы средняя выручка увеличилась не менее, чем на 2 рубля — иначе обнаруженное увеличение выручки не оправдает внедрение. Соответственно, минимальный ожидаемый эффект определим равным 2 рублям.

**Реальный эффект** - это эффект, который будет на самом деле, если мы применим наше изменение. Мы никогда не знаем, чему равен реальный эффект. Мы оцениваем его с помощью техник A/B тестирования.

Что будет, если реальный эффект окажется меньше или больше MDE? Ответ на этот вопрос наглядно показан на графике ниже:
<img src="data/img/real_effect.png" width="600">

- Если реальный эффект равен MDE, то вероятность ошибки второго рода будет в точности равна допустимой вероятности ошибки второго рода β, которую мы зафиксировали до эксперимента.
- Если реальный эффект меньше MDE, то вероятность ошибки второго рода будет больше. Это значит,
что, скорее всего, эффект мы не обнаружим.
- Если реальный эффект больше MDE, то вероятность ошибки второго рода будет меньше. Это значит,
что, скорее всего, мы обнаружим эффект.

### Запуск эксперимента

Когда запускать эксперимент? Особых ограничений на время запуска нет, но есть несколько рекомендаций, когда эксперимент начинать не стоит. 

Если есть информация о событиях, которые могут сильно повлиять на ход экспермента, то лучше запустить эксперимент после их окончания.

Примером такого события может быть большой праздник, например, новый год. Во время праздника, поведение людей отличается от их обычного поведения, это вносит дополнительные искажения в данные и затрудняет обнаружение эффекта. В этом примере есть исключение. Если изменение дизайна связано с событием (новым годом), то эксперимент естественно нужно проводить.

Желательно не запускать эксперимент в пятницу вечером, если вы не хотите задерживаться до ночи или работать в выходные. В новой функциональности могут быть ошибки, в этом случае придётся экстренно останавливать эксперимент и вносить правки.

### Продолжительность эксперимента

Чем дольше идет эксперимент, тем больше данных мы получим. Чем больше данных, тем точнее оценка эффекта. Но ждать очень долго мы не можем, нужно искать баланс. 

Важным фактором в выборе продолжительности эксперимента является наличие периодичности в действиях пользователей, так как эксперимент может оказывать разное влияние в течение одного периода. Например, из-за эксперимента выручка может увеличиться в будни и уменьшиться в выходные. Если мы проведём эксперимент с понедельника по пятницу, то увидим положительный эффект, хотя на самом деле средняя выручка за неделю не изменилась.

В поведении людей, как правило, есть недельная периодичность, поэтому продолжительность эксперимента лучше делать равной целому числу недель, например, 7 или 14 дней. 

### Подбор групп

Допустим, мы определили размер групп. Как выбрать, кто попадет в контрольную группу, а кто - в экспериментальную?

Если мы заранее знаем, какие пользователи могут участвовать в эксперименте, то можно опредлеить состав групп до начала эксперимента. Например, мы хотим отправить письмо с рекламой. У нас есть база email-адресов наших покупателей, которым мы можем отрпвить письма. Случайным образом выбираем нужное количество адресов для контрольной и экспериментальной групп. Группы готовы.

При тестировании изменений на сайте мы не знаем заранее, кто будет участвовать в эксперименте. Не все пользователи заходят на сайт каждую неделю; регулярно появляются новые пользователи, которых не было до начала эксперимента. 

Когда пользователь заходит на сайт, нам нужно определить, участвует ли он в текущем эксперименте и, если участвует, то в какой группе находитс, контрольной или экспериментальной. Если распределять запросы пользователей по группам случайным образо, то может получиться, что один пользователь, открыв сайт два раза подряд, окажется в разных группах. В первый раз он увидит старую версию сайта, а во второй раз - новую. Надо избегать такой ситуаци, потому что пользователь будет дезориентирован.

Для определения группы пользователя в онлайне (на сайте) нужно сначала идентифицировать пользователя, а затем определять его группу.  Важно, чтобы в дальнейшем он каждый раз оказывался в одной и той же группе в рамках одного эксперимента. Существует алгоритм, который позволяет быстро распределять пользователей на произвольное число групп примерно одинакового размера. Эти группы принято называть бакетами.  Мы подробно поговорим о разбиении пользователей в онлайне в десятом уроке. Сейчас отметим только два момента, которые уже актуальны для наших задач.

Если покупатель попал в экспериментальную группу в одном эксперименте, это не значит, что он должен быть в экспериментальной группе во всех последующих экспериментах. Чтобы перемешивать состав групп между экспериментами, к идентификаторам пользователей добавляется так называемая соль. **Соль** — случайная последовательность символов, которая генерируется при запуске эксперимента.

<img src="data/img/salt.png" width="600">

На нашей платформе реализация разбиения пользователей на группы следующая:

1. Для нового эксперимента создаётся уникальная соль; 
2. Вычисляется целевой размер группы по формуле для Sample Size; 
3. Вычисляется количество бакетов теста по формуле: n_buckets = ближайшее целое к n_users / sample_size снизу. Например, если n_users = 100, sample_size = 26, то 100 / 26 = 3.846, тогда объявляем n_buckets = 3;
4. С помощью соли каждому пользователю мы ставим в соответствие случайное число от 1 до n_buckets; 
5. Пользователи, которым соответствует число 1, составляют тестовую группу; контрольная группа наполняется пользователями, которым назначено число 2. Остальные пользователи в данном эксперименте не участвуют.

## Тестирование дизайна

После подготовки основных параметров эксперимента, необходимо убедиться в их корректности.

Проверить правильность дизайна можно с помощью синтетических экспериментов на исторических данных. Эксперименты называются синтетическими, потому что они не проводятся на самом деле, а моделируются численно. 

### A/A эксперимент на исторических данных

Мы можем провести неограниченное число экспериментов задним числом (за прошедший период времени? например - неделю назад), при этом используя различные способы разбиения людей на группы.

В этом случае, мы знаем, что эффекта на самом деле нет, так как мы не запускали эксперимент на прошлой неделе и группы были выбраны случайным образом. Доля экспериментов, в которых был обнаружен значимый эффект — это оценка вероятности ошибки первого рода. Если в дизайне эксперимента уровень значимости равен 0.05, то из 1000 экспериментов примерно в 50 должен быть найден значимый эффект. Такие эксперименты называются А/А экспериментами, так как у нас обе группы контрольные и никакое воздействие не оказывается.

### A/B эксперимент на исторических данных

Аналогично можно оценить вероятность ошибки второго рода. Для этого нужно также провести 1000 синтетических экспериментов на исторических данных, но с одним отличием: в экспериментальную группу нужно искусственно добавить эффект, равный ожидаемому эффекту. Это можно делать разными способами, самый простой — добавить значение эффекта ко всем значениям экспериментальной группы. 

То есть, если человек из экспериментальной группы на прошлой неделе совершил покупок на 784 рубля, а ожидаемый эффект 5 рублей, то новое значение будет 789 рублей.

Далее оцениваем эффект согласно дизайну. В итоге получаем также 1000 вердиктов. Но теперь эффект на самом деле есть. Доля экспериментов, в которых значимые отличия не были найдены — это оценка вероятности ошибки второго рода. Оценка вероятности ошибки второго рода должна быть не больше допустимой вероятности ошибки второго рода, указанной в дизайне эксперимента. 

Для выбранного MDE и фиксированной мощности 0.9 (вероятность ошибки II рода 0.1) примерно в 100 случаях из 1000 ожидается ошибка второго рода, то есть критерий утверждает, что эффекта нет, хотя на самом деле эффект есть, и в 900 случаях из 1000 ожидается обнаружение эффекта.

### Распределение p-value

Для более детального анализа можно построить графики распределения p-value для синтетических А/А и A/B экспериментов. Для этого во время синтетических экспериментов нужно не только запоминать, есть значимый эффект или нет, но и запоминать значение p-value. После 1000 синтетических экспериментов получим 1000 значений p-value. 

По этим значениям нужно построить кумулятивную гистограмму. Кумулятивная гистограмма — это такой график, значение которого в точке X равно доле значений в выборке, которые меньше X. Например, если в точке 0.2 значение равно 0.3, то  в 30% синтетических экспериментов (300 из 1000) p-value было меньше 0.2.

<img src="data/img/exp_test_platform.png" width="800">

Для экспериментов без эффекта (синтетические A/A эксперименты) есть теорема (какая теорема?), согласно которой распределение p-value должно быть равномерным. То есть кумулятивная гистограмма должна быть близка к диагонали. 
Если график распредлеения сильно отклоняется от диагонали, то это сигнал о том, что в дизайне может быть ошибка и нужно его тщательно перепроверить.

Для экспериментов с эффектом (синтетические A/B) распределение p-value должно быть выпуклым вверх. Чем он более выпуклый вверх, тем лучше. 
Это говорит о том, что при наличии эффекта часто получаются малые значения p-value, а когда p-value меньше уровня значимости, тогда говорим, что есть значимый эффект. 

Ниже представлены кумулятивные гистограммы распределений, характерных для некорректных A/A-экспериментов:
<img src="data/img/cum_hist_1.png" width="600">
<img src="data/img/cum_hist_2.png" width="600">

А это пример корректного эксперимента:
<img src="data/img/cum_hist_3.png" width="600">

Также существуют специальные тесты, которые отвечают на вопрос, можем ли мы считать данное распределение равномерным. Это можно сделать с помощью теста Колмогорова-Смирнова или в упрощенном виде использовать критерий Пирсона.

Однако, в большинстве случаев ответ виден на глаз.

### Ошибки дизайна

Допустим, мы проверили дизайн эксперимента и обнаружили, что оценки вероятностей ошибок не соответствуют значениям, заявленным в дизайне. Что делать? Нужно искать причину, почему так происходит. Универсального алгоритма действий в этой ситуации нет. 

Вот несколько ошибок в дизайне:

1. Неудачно выбранная продолжительность эксперимента. Мы уже говорили, что нужно обращать внимание на периодичность в данных и брать целое число периодов для эксперимента.

2. Неправильное разбиение пользователей на группы. Из-за ошибки в технической реализации может получиться, что в контрольной группе у нас будут одни женщины, а в экспериментальной — одни мужчины. Это совсем не случайное распределение пользователей, группы могут сильно отличаться своим поведением без влияния эксперимента.

3. Зависимые данные в выборках. Многие статистические тесты, в том числе и t-test, некорректно работают, когда данные выборок зависимы. Приведём пример зависимых данных. За время эксперимента пользователи могут совершать по несколько покупок. Покупки разных пользователей считаем независимыми. Покупки одного пользователя, скорее всего, зависимы. Если использовать в качестве данных стоимости покупок по-отдельности, то получим зависимые данные в выборках.  Если же в качестве метрики использовать суммарные траты пользователя за время эксперимента, то на одного пользователя будет одно значение, данные будут независимы.

<img src="data/img/ttest_example.png" width="600">

Оценить вероятности ошибок и проверить корректность дизайна можно с помощью синтетических А/А и А/В экспериментов на исторических данных.

Оценка вероятности ошибки первого рода должна быть примерно равна уровню значимости, а оценка вероятности ошибки второго рода должна быть не больше допустимой вероятности ошибки второго рода.

При корректном дизайне распределение p-value на синтетических А/А экспериментах должно быть равномерным, а для синтетических А/В экспериментов — выпуклым вверх.

# Приложение

#### Примеры ситуаций

- [InterviewQuery: Threaded Comments - Medium](https://www.interviewquery.com/questions/threaded-comments)

## Как работать с ненормально распределеными данными?

### Удаление выборосов

Когда этот метод работает хорошо, а когда не очень?

### Логарифмирование

Посмотреть зависимость двух переменных, а также логарифмов этих переменных. Если два логарифма связаны линейно, то как связаны исходные переменные?

### Непараметрика

Какие непараметрические тесты можно использовать и в каких случаях?